# Ensemble averaging
Aug. 15. 2025. Written by W.-H. Shin \
Combining ChemProp, ESOL, and LGBM for predicting LogS.

## ChemProp
Heid et al. JCIM 2024 https://pubs.acs.org/doi/10.1021/acs.jcim.3c01250 \
Predicts molecular properties using MPNN

In [1]:
#@title Install ChemProp from GitHub if running in Google Colab
import os

if os.getenv("COLAB_RELEASE_TAG"):
    try:
        import chemprop
        %cd chemprop/examples
    except ImportError:
        !git clone https://github.com/chemprop/chemprop.git
        %cd chemprop
        !pip install .
        %cd examples

Cloning into 'chemprop'...
remote: Enumerating objects: 25403, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 25403 (delta 94), reused 38 (delta 38), pack-reused 25240 (from 3)
Receiving objects: 100% (25403/25403), 876.21 MiB | 20.75 MiB/s, done.
Resolving deltas: 100% (18214/18214), done.
Updating files: 100% (336/336), done.
/content/chemprop
Processing /content/chemprop
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.2/824.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

### Load modules and dataset

In [2]:
from pathlib import Path

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
import pandas as pd

from chemprop import data, featurizers, models, nn

In [3]:
#chemprop_dir = Path.cwd().parent
#input_path = chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv" # path to your data .csv file
%cd
!wget https://raw.githubusercontent.com/schwallergroup/ai4chem_course/main/notebooks/02%20-%20Supervised%20Learning/data/esol.csv
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['logS'] # list of names of the columns containing targets

/root
--2025-08-15 00:48:20--  https://raw.githubusercontent.com/schwallergroup/ai4chem_course/main/notebooks/02%20-%20Supervised%20Learning/data/esol.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34453 (34K) [text/plain]
Saving to: ‘esol.csv’

esol.csv            100%[===================>]  33.65K  --.-KB/s    in 0.01s   

2025-08-15 00:48:20 (3.00 MB/s) - ‘esol.csv’ saved [34453/34453]



In [4]:
df_input = pd.read_csv('esol.csv')
df_input.columns = ['smiles', 'logS']
df_input

,smiles,logS
0,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,-0.770
1,Cc1occc1C(=O)Nc2ccccc2,-3.300
2,CC(C)=CCCC(C)=CC(=O),-2.060
3,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,-7.870
4,c1ccsc1,-1.330
...,...,...
1123,FC(F)(F)C(Cl)Br,-1.710
1124,CNC(=O)ON=C(SC)C(=O)N(C)C,0.106
1125,CCSCCSP(=S)(OC)OC,-3.091
1126,CCC(C)C,-3.180


### Split dataset

In [5]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

In [6]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))  # unpack the tuple into three separate lists
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)
df_test = df_input.loc[test_indices[0], :].copy()
df_train = df_input.loc[train_indices[0], :].copy()
df_val = df_input.loc[val_indices[0], :].copy()
df_test

,smiles,logS
275,Ic1cccc2ccccc12,-4.550
806,CCC(C)C(C)C,-4.280
1084,CC(=O)OC3(CCC4C2CCC1=CC(=O)CCC1C2CCC34C)C#C,-4.800
182,Cc1ccccc1n3c(C)nc2ccccc2c3=O,-2.925
32,COC(=O)Nc1cccc(OC(=O)Nc2cccc(C)c2)c1,-4.805
...,...,...
763,Cc1ccc(C)c(C)c1,-3.310
835,c1ccccc1,-1.640
559,Clc1ccc(c(Cl)c1)c2ccc(Cl)c(Cl)c2Cl,-7.800
684,CCCCOC,-0.990


### Assinging input features

In [7]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.MoleculeDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data[0], featurizer)

In [8]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

### Construct MPNN model for regression

In [9]:
mp = nn.BondMessagePassing()
agg = nn.MeanAggregation()
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(output_transform=output_transform)

In [10]:
batch_norm = True
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
  (metrics): ModuleList(
    (0): RMSE(task_weights=[[1.0]])
    (1): MAE(task_weigh

### Training

In [11]:
# Configure model checkpointing
checkpointing = ModelCheckpoint(
    "checkpoints",  # Directory where model checkpoints will be saved
    "best-{epoch}-{val_loss:.2f}",  # Filename format for checkpoints, including epoch and validation loss
    "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
    mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True,  # Always save the most recent checkpoint, even if it's not the best
)


trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=20, # number of epochs to train for
    callbacks=[checkpointing], # Use the configured checkpoint callback
)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(mpnn, train_loader, val_loader)

INFO: Loading `train_dataloader` to estimate number of stepping batches.
INFO:lightning.pytorch.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.
INFO: 
  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | train
3 | predictor       | RegressionFFN      | 90.6 K | train
4 | X_d_transform   | Identity           | 0      | train
5 | metrics         | ModuleList         | 0      | train
---------------------------------------------------------------
318 K     Trainable params
0         Non-trainable params
318 K     Total params
1.276     Total estimated model params size (MB)
25        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name            | Type         

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


### Performance check with test set

In [13]:
results = trainer.test(dataloaders=test_loader)

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:149: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO: Restoring states from the checkpoint path at /root/checkpoints/best-epoch=19-val_loss=0.08.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /root/checkpoints/best-epoch=19-val_loss=0.08.ckpt
INFO: Loaded model weights from the checkpoint at /root/checkpoints/best-epoch=19-val_loss=0.08.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /root/checkpoints/best-epoch=19-val_loss=0.08.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/mae          │    0.4918281137943268     │
│         test/rmse         │    0.6689251065254211     │
└───────────────────────────┴───────────────────────────┘

### Load the best model and check it for external validation set

In [14]:
checkpoint_path = checkpointing.best_model_path
print(checkpoint_path)
best_model = models.MPNN.load_from_checkpoint(checkpoint_path)

/root/checkpoints/best-epoch=19-val_loss=0.08.ckpt


In [15]:
import torch
import numpy as np

with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(best_model, test_loader)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [16]:
test_preds = np.concatenate(test_preds, axis=0)
df_test['ChemProp'] = test_preds
df_test

,smiles,logS,ChemProp
275,Ic1cccc2ccccc12,-4.550,-4.804772
806,CCC(C)C(C)C,-4.280,-4.077079
1084,CC(=O)OC3(CCC4C2CCC1=CC(=O)CCC1C2CCC34C)C#C,-4.800,-4.715652
182,Cc1ccccc1n3c(C)nc2ccccc2c3=O,-2.925,-3.120181
32,COC(=O)Nc1cccc(OC(=O)Nc2cccc(C)c2)c1,-4.805,-3.812439
...,...,...,...
763,Cc1ccc(C)c(C)c1,-3.310,-3.268891
835,c1ccccc1,-1.640,-1.042952
559,Clc1ccc(c(Cl)c1)c2ccc(Cl)c(Cl)c2Cl,-7.800,-7.354958
684,CCCCOC,-0.990,-1.064527


## ESOL
ESOL is a simple regression model for estimating solubility. \
Delaney J. Chem. Inf. Comput. Sci. 2004 https://pubs.acs.org/doi/10.1021/ci034243x

In [17]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski

def calc_ap(mol):
  aromatic_query = Chem.MolFromSmarts("a")
  matches = mol.GetSubstructMatches(aromatic_query)
  ap = len(matches) / mol.GetNumAtoms()
  return ap

def calc_esol_descriptors(mol):
  mw = Descriptors.MolWt(mol)
  logp = Crippen.MolLogP(mol)
  rotors = Lipinski.NumRotatableBonds(mol)
  return mw, logp, rotors

def calc_esol(mol):
  mw, logp, rotors = calc_esol_descriptors(mol)
  ap = calc_ap(mol)
  esol = 0.16 - 0.63 * logp -0.0062 * mw + 0.066 * rotors - 0.74 * ap
  return esol

esol = df_test.apply(lambda x: calc_esol(Chem.MolFromSmiles(x['smiles'])), axis=1)
df_test['ESOL'] = esol
df_test

,smiles,logS,ChemProp,ESOL
275,Ic1cccc2ccccc12,-4.550,-4.804772,-4.257933
806,CCC(C)C(C)C,-4.280,-4.077079,-2.023026
1084,CC(=O)OC3(CCC4C2CCC1=CC(=O)CCC1C2CCC34C)C#C,-4.800,-4.715652,-4.444750
182,Cc1ccccc1n3c(C)nc2ccccc2c3=O,-2.925,-3.120181,-3.840624
32,COC(=O)Nc1cccc(OC(=O)Nc2cccc(C)c2)c1,-4.805,-3.812439,-4.291642
...,...,...,...,...
763,Cc1ccc(C)c(C)c1,-3.310,-3.268891,-2.724014
835,c1ccccc1,-1.640,-1.042952,-2.126865
559,Clc1ccc(c(Cl)c1)c2ccc(Cl)c(Cl)c2Cl,-7.800,-7.354958,-6.491240
684,CCCCOC,-0.990,-1.064527,-1.091257


## LightGBM
Same protocol as previous tutorial

### Setup environment

In [18]:
!pip install lightgbm

In [19]:
from tqdm.auto import tqdm
from rdkit.Chem import rdMolDescriptors
from lightgbm import LGBMRegressor
from rdkit.rdBase import BlockLogs

In [20]:
tqdm.pandas()

### Property calculation

In [21]:
property_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
property_getter = rdMolDescriptors.Properties(property_names)

In [22]:
def smi2props(smi):
    mol = Chem.MolFromSmiles(smi)
    props = None
    if mol:
        Chem.DeleteSubstructs(mol, Chem.MolFromSmarts("[#1X0]"))
        props = np.array(property_getter.ComputeProperties(mol))
    return props

In [23]:
block = BlockLogs() # To remove long logs
df_train['props'] = df_train.smiles.progress_apply(smi2props)
del block

  0%|          | 0/902 [00:00<?, ?it/s]

In [24]:
df_train = df_train[df_train['props'].notna()]
df_train[property_names] = df_train['props'].to_list()
df_train

,smiles,logS,props,exactmw,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
427,C1SC(=S)NC1(=O),-1.770,"[132.965605716, 133.197, 2.0, 1.0, 0.0, 1.0, 3...",132.965606,133.197,2.0,1.0,0.0,1.0,3.0,...,3.431852,1.754601,0.575839,0.575839,0.317099,-0.09,5.054718,1.794713,1.138114,1.295967
638,CCBr,-1.090,"[107.95746226, 108.966, 0.0, 0.0, 0.0, 0.0, 0....",107.957462,108.966,0.0,0.0,0.0,0.0,0.0,...,2.085071,0.974368,0.000000,0.000000,0.000000,0.48,3.480000,2.480000,2.480000,2.876800
950,CC(C)OC(=O)Nc1cccc(Cl)c1,-3.380,"[213.055656304, 213.66400000000002, 3.0, 1.0, ...",213.055656,213.664,3.0,1.0,2.0,1.0,2.0,...,8.081212,4.309001,1.413758,1.413758,0.927625,-1.22,10.858247,4.861820,3.661951,3.770775
436,CCSCC,-1.340,"[90.05032132, 90.191, 0.0, 0.0, 2.0, 0.0, 1.0,...",90.050321,90.191,0.0,0.0,2.0,0.0,1.0,...,3.822462,1.991564,0.408248,0.408248,0.204124,0.35,5.350000,4.350000,4.350000,4.654500
1122,Cc1cccc(c1)N(=O)=O,-2.440,"[137.047678464, 137.138, 3.0, 0.0, 1.0, 0.0, 2...",137.047678,137.138,3.0,0.0,1.0,0.0,2.0,...,5.573111,2.910122,1.212475,1.212475,0.741533,-1.38,6.736009,2.473197,1.312840,1.665948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,CCCCc1c(C)nc(NCC)nc1OS(=O)(=O)N(C)C,-4.160,"[316.156911628, 316.4270000000001, 7.0, 1.0, 8...",316.156912,316.427,7.0,1.0,8.0,1.0,6.0,...,13.303061,7.046873,2.922625,2.922625,1.691525,-1.41,17.641046,7.556273,4.817297,6.347646
547,Clc1ccc(c(Cl)c1)c2cc(Cl)ccc2Cl,-6.570,"[289.922360912, 291.99199999999996, 0.0, 0.0, ...",289.922361,291.992,0.0,0.0,1.0,0.0,0.0,...,7.975960,4.481997,2.149711,2.149711,1.411806,-0.40,12.067412,4.848492,2.464317,3.656797
579,c1ccc(cc1)c2ccc(cc2)c3ccccc3,-7.110,"[230.109550448, 230.31, 0.0, 0.0, 2.0, 0.0, 0....",230.109550,230.310,0.0,0.0,2.0,0.0,0.0,...,10.082904,6.142734,3.099145,3.099145,2.116334,-2.34,10.791413,4.886590,2.184220,2.929623
541,CC(=O)C1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3CCC21C,-3.817,"[330.21949481999997, 330.4680000000001, 3.0, 1...",330.219495,330.468,3.0,1.0,1.0,1.0,3.0,...,15.229966,9.695734,8.695991,8.695991,7.153613,-0.96,16.505310,5.030469,1.880066,3.459560


### Build and LGBM

In [25]:
train_X = df_train[property_names]
train_y = df_train['logS']

In [26]:
lgbm = LGBMRegressor()
lgbm.fit(train_X, train_y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000638 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5502
[LightGBM] [Info] Number of data points in the train set: 902, number of used features: 41
[LightGBM] [Info] Start training from score -3.049081


LGBMRegressor()

### Validation on test set

In [27]:
block = BlockLogs() # To remove long logs
df_test['props'] = df_test.smiles.progress_apply(smi2props)
del block

  0%|          | 0/114 [00:00<?, ?it/s]

In [28]:
df_test = df_test[df_test['props'].notna()]
df_test[property_names] = df_test['props'].to_list()

In [29]:
test_X = df_test[property_names]
lgbm_pred = lgbm.predict(test_X)
df_test['LGBM'] = lgbm_pred
df_test = df_test[['smiles', 'logS', 'ChemProp', 'ESOL', 'LGBM']]
df_test

,smiles,logS,ChemProp,ESOL,LGBM
275,Ic1cccc2ccccc12,-4.550,-4.804772,-4.257933,-4.502030
806,CCC(C)C(C)C,-4.280,-4.077079,-2.023026,-3.834139
1084,CC(=O)OC3(CCC4C2CCC1=CC(=O)CCC1C2CCC34C)C#C,-4.800,-4.715652,-4.444750,-4.531011
182,Cc1ccccc1n3c(C)nc2ccccc2c3=O,-2.925,-3.120181,-3.840624,-4.050238
32,COC(=O)Nc1cccc(OC(=O)Nc2cccc(C)c2)c1,-4.805,-3.812439,-4.291642,-4.499372
...,...,...,...,...,...
763,Cc1ccc(C)c(C)c1,-3.310,-3.268891,-2.724014,-3.441113
835,c1ccccc1,-1.640,-1.042952,-2.126865,-1.795502
559,Clc1ccc(c(Cl)c1)c2ccc(Cl)c(Cl)c2Cl,-7.800,-7.354958,-6.491240,-7.489063
684,CCCCOC,-0.990,-1.064527,-1.091257,-1.229612


## Ensemble averaging

In [30]:
df_test['Ensemble'] = df_test['ChemProp'] + df_test['ESOL'] + df_test['LGBM']
df_test['Ensemble'] /= 3
df_test

,smiles,logS,ChemProp,ESOL,LGBM,Ensemble
275,Ic1cccc2ccccc12,-4.550,-4.804772,-4.257933,-4.502030,-4.521579
806,CCC(C)C(C)C,-4.280,-4.077079,-2.023026,-3.834139,-3.311415
1084,CC(=O)OC3(CCC4C2CCC1=CC(=O)CCC1C2CCC34C)C#C,-4.800,-4.715652,-4.444750,-4.531011,-4.563804
182,Cc1ccccc1n3c(C)nc2ccccc2c3=O,-2.925,-3.120181,-3.840624,-4.050238,-3.670348
32,COC(=O)Nc1cccc(OC(=O)Nc2cccc(C)c2)c1,-4.805,-3.812439,-4.291642,-4.499372,-4.201151
...,...,...,...,...,...,...
763,Cc1ccc(C)c(C)c1,-3.310,-3.268891,-2.724014,-3.441113,-3.144672
835,c1ccccc1,-1.640,-1.042952,-2.126865,-1.795502,-1.655106
559,Clc1ccc(c(Cl)c1)c2ccc(Cl)c(Cl)c2Cl,-7.800,-7.354958,-6.491240,-7.489063,-7.111754
684,CCCCOC,-0.990,-1.064527,-1.091257,-1.229612,-1.128465


In [31]:
from sklearn.metrics import mean_squared_error

y_true = df_test['logS']
y_ens = df_test['Ensemble']
y_chemprop = df_test['ChemProp']
y_esol = df_test['ESOL']
y_lgbm = df_test['LGBM']

mse_chemprop = mean_squared_error(y_true, y_chemprop)
mse_esol = mean_squared_error(y_true, y_esol)
mse_lgbm = mean_squared_error(y_true, y_lgbm)
mse_ens = mean_squared_error(y_true, y_ens)

print(f"MSE ChemProp: {mse_chemprop}")
print(f"MSE ESOL: {mse_esol}")
print(f"MSE LGBM: {mse_lgbm}")
print(f"MSE Ensemble: {mse_ens}")

MSE ChemProp: 0.44746080191298243
MSE ESOL: 1.3172918594691903
MSE LGBM: 0.38179905242563483
MSE Ensemble: 0.4648465800485896


In [32]:
from sklearn.metrics import r2_score

r2_chemprop = r2_score(y_true, y_chemprop)
r2_esol = r2_score(y_true, y_esol)
r2_lgbm = r2_score(y_true, y_lgbm)
r2_ens = r2_score(y_true, y_ens)

print(f"R2 ChemProp: {r2_chemprop}")
print(f"R2 ESOL: {r2_esol}")
print(f"R2 LGBM: {r2_lgbm}")
print(f"R2 Ensemble: {r2_ens}")

R2 ChemProp: 0.9023113541907268
R2 ESOL: 0.7124117747588765
R2 LGBM: 0.9166464811146136
R2 Ensemble: 0.8985157298250909
